# Running Native Functions

Two of the previous notebooks showed how to [execute semantic functions inline](./03-running-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

* Writing logic around how to run a prompt that changes the prompt's outcome.
* Using external data sources to gather data to concatenate into your prompt.
* Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the Python framework.

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import semantic_kernel as sk
from semantic_kernel.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.config.add_text_backend("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_backend("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

Let's create a function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.

For this example, we will create the following directory structure:

```txt
project-root/
├── __main__.py
└── skills/
    └── NumberGenerator/
        └── native_function.py
    └── WriterSkill/
        └── CorgiStory/
            └── skprompt.txt
            └── config.json
```

First, let's create our native function.
Add the following code to native_function.py

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumber"
    )
    def generate_number(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, input)) 
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$INPUT}}` and generates that number of paragraphs about two Corgis on an adventure.

Add the following prompt to our `skprompt.txt` file.

```
Write a short story about two Corgis on an adventure.
The storey must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
```

...and update the config.json file

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a story about two Corgis",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  },
  "default_backends": [
    "text-davinci-003"
  ]
}
```

Finally, we're ready to setup our script to execute the functions. Open the `__main__.py` file and add the following code.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(input):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_backend("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))

    # Import native skill
    number_generator = kernel.import_native_skill_from_directory(base_skills_directory, "NumberGenerator"),

    # Import semantic skill
    story_generator = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill")

    # Run the number generator
    number_result = number_generator.invoke(input=input)

    # Pass the output to the semantic story function
    story_generator.invoke(input=number_result)

    print(story_generator)

    
if __name__ == "__main__":
    if len(sys.argv) <= 1:
        print("This script requires one input, which should be an integer. Example: `python __main__.py 8`")
    else:
        main(sys.argv[1])

Run the script with `python __main__.py 8` and it will create a story between 3-8 paragraphs.

## Context Variables

Let's expand on our example to add a second input to both the native and the semantic functions.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the NumberGenerator function will pick a number between the first and second input.

First, let's update our `__main__.py` file to accept the user input and pass the variables to the native and semantic functions. This will leverage the `semantic_kernel.ContextVariables` class.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(min: str, max: str, language: str):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_backend("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))

    # Import native skill
    number_generator = kernel.import_native_skill_from_directory(base_skills_directory, "NumberGenerator"),

    # Import semantic skill
    story_generator = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill")


    context_variables = sk.ContextVariables(variables={
        "min": min,
        "max": max,
        "language": language,
        "paragraph_count": ""
    })

    # Run the number generator
    context_variables['paragraph_count'] = number_generator.invoke(variables=context_variables)

    # Pass the output to the semantic story function
    story_generator.invoke(variables=context_variables)

    print(story_generator)

    
if __name__ == "__main__":
    if len(sys.argv) <= 3:
        print("This script requires three inputs. The first two are min, max, which should be integers. The third is the language to write in. Example: `python __main__.py 4 10 english`")
    else:
        main(sys.argv[1], sys.argv[2], sys.argv[3])

Now that we can accept additional inputs from the user and we have those wired up to the functions, we need to update those functions to accept these new arguments.

Let's start with the native function.

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function
from semantic_kernel import SKContext

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"]))) 
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

Finally, we'll update our semantic function to handle the new inputs.

```
Write a short story about two Corgis on an adventure.
The storey must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
```

Great! We're all set, give it a test: 

```bash
python __main__.py 4 10 english
```

## Calling Semantic Functions from Native Functions with Context Variables

One more example to show here is how we can modify this code to make the native function directly call the semantic function.

Note that in this case, we'd likely want to rename our native function to better represent the intent of producing the story, not just generating a number. However, for simplicity of example, we'll leave the function names as-is.

First, let's trim down our `__main__.py` file to remove the calls to our semantic function.

In [ ]:
import os
import sys
import semantic_kernel as sk
from semantic_kernel.ai.open_ai import OpenAITextCompletion, AzureTextCompletion


def main(min: str, max: str, language: str):
    # Setup Kernel
    kernel = sk.Kernel()
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_backend("text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id))

    base_skills_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), "./skills/"))

    # Import native skill
    number_generator = kernel.import_native_skill_from_directory(base_skills_directory, "NumberGenerator"),

    # Import semantic skill.
    # This still needs to get imported into the kernel even if we're not calling it in this file.
    story_generator = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterSkill")


    context_variables = sk.ContextVariables(variables={
        "min": min,
        "max": max,
        "language": language,
        "paragraph_count": ""
    })

    # Run the number generator
    story_generator = number_generator.invoke(variables=context_variables)

    print(story_generator)

    
if __name__ == "__main__":
    if len(sys.argv) <= 3:
        print("This script requires three inputs. The first two are min, max, which should be integers. The third is the language to write in. Example: `python __main__.py 4 10 english`")
    else:
        main(sys.argv[1], sys.argv[2], sys.argv[3])

Next, we'll update our native function to call our semantic function and return the value of the semantic function's output.

In [ ]:
import random
from semantic_kernel.skill_definition import sk_function
from semantic_kernel import SKContext

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            context['paragraph_count'] = str(random.randint(int(context["min"]), int(context["max"]))) 
            return context.skills.get_semantic_function("WriterSkill", "CorgiStory").invoke(variables=context.variables)
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e